In [1]:
#Looking to determine whether a given mesh has any overlapping data.
#Basically, if we convert a mesh into an RGBD image, is there any risk of losing information.
#The way I'm looking at testing this is to consider a each triangle in the mesh and test:
##Is there any vertices, that are within the shadow of this triangle, along the z-axis

In [2]:
import open3d as o3d
import numpy as np

In [3]:
#Data Path
path = "../../../Data/Tiles/"

In [4]:
from os import listdir
from os.path import isfile, join

allfiles = [f for f in listdir(path) if isfile(join(path, f))]
allObjNames = [f for f in allfiles if (f.find(".obj") != -1)]

In [5]:
index = 1
mesh = o3d.io.read_triangle_mesh(path + allObjNames[index],True)

[Open3D INFO] Skipping non-triangle primitive geometry of type: 1
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2


In [6]:
#Looking at the example image
o3d.visualization.draw_geometries([mesh])

In [7]:
#Looking at available data in object
dir(mesh)

['HalfEdgeTriangleMesh',
 'Image',
 'LineSet',
 'PointCloud',
 'RGBDImage',
 'TetraMesh',
 'TriangleMesh',
 'Type',
 'Unspecified',
 'VoxelGrid',
 '__add__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'adjacency_list',
 'clear',
 'cluster_connected_triangles',
 'compute_adjacency_list',
 'compute_convex_hull',
 'compute_triangle_normals',
 'compute_vertex_normals',
 'create_arrow',
 'create_box',
 'create_cone',
 'create_coordinate_frame',
 'create_cylinder',
 'create_from_point_cloud_alpha_shape',
 'create_from_point_cloud_ball_pivoting',
 'create_from_point_cloud_poisson',
 'create_icosahedron',
 'create_moebius',
 'create_octahedron',
 'create_sph

In [8]:
#Checking 3rd column is z-axis
#E.g. low standard deviation impies little vertical change
np.std(np.asarray(mesh.vertices)[:,2])

6.588084719727278

In [9]:
#Function (and utility functions) for determining if point within triangle
#point is defined as list of size 2 [x,y], triangle is
#defined as a list of size 3, each a list of size 2 defining a point
def tri_order(triangle):
    #Ordering points in triangle by x order
    xs = [a[0] for a in triangle]
    order_ind = np.argsort(xs)
    return [triangle[a] for a in order_ind]

def line_cond(m,c,g,point):
    return ((point[1]-m*point[0]>c) == g)


def in_Triangle(point, triangle):
    a,b,c = tri_order(triangle)
    
    m_ab = (b[1]-a[1])/(b[0]-a[0])
    c_ab = a[1] - m_ab*a[0]
    g_ab = (c[1]>(m_ab*c[0]+c_ab))
    if not line_cond(m_ab,c_ab,g_ab,point):
        return False
    
    m_bc = (b[1]-c[1])/(b[0]-c[0])
    c_bc = b[1] - m_bc*b[0]
    g_bc = (a[1]>(m_bc*a[0]+c_bc))
    if not line_cond(m_bc,c_bc,g_bc,point):
        return False
    
    m_ac = (a[1]-c[1])/(a[0]-c[0])
    c_ac = a[1] - m_ac*a[0]
    g_ac = (b[1]>(m_ac*b[0]+c_ac))
    if not line_cond(m_ac,c_ac,g_ac,point):
        return False
    
    return True

In [10]:
#Testing
tri = [[0.01,0],[0,1],[1,0]]
point = [0.8,0.2]
#in_Triangle([point],tri)

In [11]:
#Triangle function for lists of points
#Triangle has the same definintion
#Point is now a list of points

def tri_order(triangle):
    #Ordering points in triangle by x order
    xs = [a[0] for a in triangle]
    order_ind = np.argsort(xs)
    return [triangle[a] for a in order_ind]

def line_cond(m,c,g,point):
    return ((point[1]-m*point[0]>c) == g)


def in_Triangle(points, triangle):
    a,b,c = tri_order(triangle)
    
    fact_table = [1]*len(points)
    
    m_ab = (b[1]-a[1])/(b[0]-a[0])
    c_ab = a[1] - m_ab*a[0]
    g_ab = (c[1]>(m_ab*c[0]+c_ab))
    
    m_bc = (b[1]-c[1])/(b[0]-c[0])
    c_bc = b[1] - m_bc*b[0]
    g_bc = (a[1]>(m_bc*a[0]+c_bc))
    
    m_ac = (a[1]-c[1])/(a[0]-c[0])
    c_ac = a[1] - m_ac*a[0]
    g_ac = (b[1]>(m_ac*b[0]+c_ac))
    
    for a in range(len(points)):
        if not line_cond(m_ab,c_ab,g_ab,points[a]):
            fact_table[a] = 0
            continue
        if not line_cond(m_bc,c_bc,g_bc,points[a]):
            fact_table[a] = 0
            continue
        if not line_cond(m_ac,c_ac,g_ac,points[a]):
            fact_table[a] = 0
            continue    
    return fact_table

In [12]:
all_verts = np.asarray(mesh.vertices)[:,0:2].tolist()
tri_inds = np.asarray(mesh.triangles).tolist()
triangles = [[all_verts[b] for b in range(len(tri_inds[a]))] for a in range(len(tri_inds))]

In [14]:
from time import time
n_iter = 1000
start = time()
[sum(in_Triangle(all_verts,triangles[a])) for a in range(n_iter)]
end = time()
print(n_iter," took ",end-start," seconds to complete.")
img_time_est = len(triangles)*(end-start)/n_iter
print("A single image would take ",img_time_est," seconds to complete.")
all_img_time = len(allObjNames)*img_time_est
print("All images would take",all_img_time," seconds to complete.")

1000  took  18.577357530593872  seconds to complete.
A single image would take  3134.7247323548795  seconds to complete.
All images would take 2401199.144983838  seconds to complete.


In [17]:
mesh.compute_adjacency_list()
mesh.adjacency_list

[{1, 2, 703, 1055, 1843},
 {0, 2, 703, 744, 1857, 2049, 2201},
 {0, 1, 642, 744, 1055},
 {4, 5, 2455, 10195, 10526},
 {3, 5, 2407, 2413, 2454, 2455, 2883},
 {3, 4, 2407, 10519, 10526},
 {7, 8, 5256, 5772, 6763},
 {6, 8, 255, 5256, 5257, 5752, 5753, 6769},
 {6, 7, 255, 257, 5318, 5985, 6466, 6763},
 {10, 11, 3642, 3766, 3855, 3856},
 {9, 11, 1309, 3766},
 {9, 10, 1309, 3642, 3643, 3768, 3769, 7740},
 {13, 14, 5527, 5528, 7500, 7528},
 {12, 14, 5925, 5926, 6243, 7528},
 {12, 13, 5918, 6241, 6243, 7500},
 {16, 17, 4025, 4039, 4376, 4754, 5619},
 {15, 17, 4049, 4050, 4170, 4370, 4468, 4469, 4754},
 {15, 16, 4024, 4025, 4469, 4474},
 {19, 20, 1475, 2018, 2207},
 {18, 20, 1475, 1486, 1686},
 {18, 19, 1486, 1766, 2195, 2207},
 {22, 23, 443, 778, 783, 1947},
 {21, 23, 111, 112, 773, 774, 778},
 {21, 22, 112, 775, 777, 889, 1947, 2013, 2023, 8901},
 {25, 26, 71, 1467, 1469, 1471, 7990, 9110},
 {24, 26, 71, 1897, 1942, 2298, 93440},
 {24, 25, 1467, 1468, 1897},
 {28, 29, 4095, 4123, 4554, 4676},